In [401]:
import numpy as np
import pandas as pd
import math

In [402]:
# Вывод матрицы на экран
def print_arr( string, namevec, a ):
    if (type(a) == int) or (type(a) == float):
        print(a)
    else:
        print( string )
        for k in range(len(a)):   
            print("{}[{}] = {:8.4f}".format(namevec, k, a[k]))

# Проверка 3х-диаг. матрицы коэффициентов на корректность
def isCorrectArray(a):
    n = len(a)
    
    for row in range(0, n):
        if( len(a[row]) != n ):
            print('Не соответствует размерность')
            return False
        
    for row in range(1, n - 1):
        if(abs(a[row][row]) < abs(a[row][row - 1]) + abs(a[row][row + 1])):
            print('Не выполнены условия достаточности')
            return False
 
    if (abs(a[0][0]) < abs(a[0][1]))or(abs(a[n - 1][n - 1]) < abs(a[n - 1][n - 2])):
        print('Не выполнены условия достаточности')
        return False
        
    
    for row in range(0, len(a)):
        if( a[row][row] == 0 ):
            print('Нулевые элементы на главной диагонали')
            return False
    return True

# Процедура нахождения решения 3-х диагональной матрицы
def solution(a, b):
    if( not isCorrectArray(a) ):
        print('Ошибка в исходных данных')
        return -1 
 
    n = len(a)
    x = np.array([.0 for k in range(0, n)]) # обнуление вектора решений
    # print('Размерность матрицы: ',n,'x',n)
    
    # Прямой ход
    v = np.array([.0 for k in range(0, n)])
    u = np.array([.0 for k in range(0, n)])
    # для первой 0-й строки
    v[0] = a[0][1] / (-a[0][0]) 
    u[0] = ( - b[0]) / (-a[0][0]) 
    for i in range(1, n - 1): # заполняем за исключением 1-й и (n-1)-й строк матрицы
        v[i] = a[i][i+1] / ( -a[i][i] - a[i][i-1]*v[i-1] )
        u[i] = ( a[i][i-1]*u[i-1] - b[i] ) / ( -a[i][i] - a[i][i-1]*v[i-1] )
    # для последней (n-1)-й строки
    v[n-1] = 0
    u[n-1] = (a[n-1][n-2]*u[n-2] - b[n-1]) / (-a[n-1][n-1] - a[n-1][n-2]*v[n-2])
    
    
    # Обратный ход
    x[n-1] = u[n-1]
    for i in range(n-1, 0, -1):
        x[i-1] = v[i-1] * x[i] + u[i-1]
        
    return x

In [403]:
# параметры
C_t = 7
mu = 2
eps = 10 ** -4

# модельная задача №1
# аналитическое решение
def u_analytical_1(t, r, phi):
    return r*np.sin(phi)/math.sqrt(C_t-6*t) 

# начальное условие
def u_init(r, phi):
    return r * np.sin(phi) / math.sqrt(7)
# граничное условие при r = 0
def u_bound_r_left(t, phi):
    return 0

# граничное условие при r = 1
def u_bound_r_right(t, phi):
    return np.sin(phi)/math.sqrt(7 - 6*t)

# граничное условие при phi = 0
def u_bound_phi_left(t, r):
    return 0

# граничное условие при phi = pi / 2
def u_bound_phi_right(t, r):
    return r/math.sqrt(C_t-6*t)

In [404]:
# прогонка №1
def a_l(u_prev, u, u_next, r_prev, r, r_next, tau, h_r):
    return -(r_next)**2 * (u_next ** mu + u ** mu) * tau / (2 * (r **2) * (h_r ** 2))

def c_l(u_prev, u, u_next, r_prev, r, r_next, tau, h_r):
    return -(r_prev)**2 * (u ** mu + u_prev ** mu) * tau / (2 * (r **2) * (h_r ** 2))

# прогонка №2
def a_m(u_prev, u, u_next, r, tau, h_phi, phi, phi_next, phi_prev):
    return -np.sin(phi_next)*(u_next ** mu + u ** mu) * tau / (2 * (r ** 2) * (h_phi ** 2)*np.sin(phi))

def c_m(u_prev, u, u_next, r, tau, h_phi, phi, phi_next, phi_prev):
    return -np.sin(phi_prev)*(u ** mu + u_prev ** mu) * tau / (2 * (r ** 2) * (h_phi ** 2)*np.sin(phi))

In [405]:
def progonka(koef_L, koef_M, N):
    # сетка
    # радиальная часть
    L_0 = 6
    
    L = koef_L * (L_0 - 1) + 1
    print("L = ", L)
    r_grid, h_r = np.linspace(0, 1, L, retstep=True)
    r_grid_0 = np.linspace(0, 1, L_0)  # сетка для печати
    
    # угловая часть
    M_0 = 6
   
    M = koef_M * (M_0 - 1) + 1
    print("M = ", M)
    phi_grid, h_phi = np.linspace(0, np.pi / 2, M, retstep=True)
    phi_grid_0 = np.linspace(0, np.pi / 2, M_0)  # сетка для печати
    
    # временнaя часть
    T = 1  # окончание
    t_grid, tau = np.linspace(0, T, N, retstep=True)
    
    # сеточная функция
    u = np.full((L, M), np.nan)
    # задание начального условия
    for l in range(L):
        for m in range(M):
            u[l, m] = u_init(r_grid[l], phi_grid[m])
    u_prev = []
    u_k    = []
    u_tmp  = []
    # print(f"u = \n{u}")
    k = 0
    # цикл по времени
    for t in t_grid[1:]:
        # переносим текущий слой в предыдущий, обнуляем текущий слой
        u_prev = u.copy()
        k += 1
        # цикл по k
        flag_k = True
        while flag_k:
            
            u_k   = u.copy()
            u     = np.full((L, M), np.nan)
            u_tmp = np.full((L, M), np.nan)
            # первая прогонка
            for m in range(1, M - 1, 1):
                # print(f"    m = {m}")
                A = np.full((L, L), .0)
                A[ 0,  0] = 1
                A[-1, -1] = 1
                d = np.full(L, .0)
                # print(f"d = \n{d}")
                d[ 0] = u_bound_r_left (t, phi_grid[m])
                d[-1] = u_bound_r_right(t, phi_grid[m])
                # print(f"d = \n{d}")
                for l in range(1, L - 1, 1):
                    r_prev = (r_grid[l - 1] + r_grid[l]) / 2
                    r_next = (r_grid[l + 1] + r_grid[l]) / 2
                    
                    A[l, l - 1] = c_l(u_k[l - 1, m], u_k[l, m], u_k[l + 1, m], r_prev, r_grid[l], r_next, tau, h_r)
                    A[l, l + 1] = a_l(u_k[l - 1, m], u_k[l, m], u_k[l + 1, m], r_prev, r_grid[l], r_next, tau, h_r)
                    A[l, l] = 1 - A[l, l - 1] - A[l, l + 1]
                    d[l]        = u_prev[l, m]
                sol = solution(A, d)

                u_tmp[:, m] = sol

            u_tmp[:,  0] = u_bound_phi_left (t, r_grid)
            u_tmp[:, -1] = u_bound_phi_right(t, r_grid)

    
            # вторая прогонка
            for l in range(1, L - 1, 1):
                A = np.full((M, M), .0)
                A[ 0,  0] = 1
                A[-1, -1] = 1
                d = np.full(L, .0)
                d[ 0] = u_bound_phi_left (t, r_grid[l])
                d[-1] = u_bound_phi_right(t, r_grid[l])
                for m in range(1, M - 1, 1):
                    phi_prev = (phi_grid[m - 1] + phi_grid[m]) / 2
                    phi_next = (phi_grid[m + 1] + phi_grid[m]) / 2
                    A[m, m - 1] = c_m(u_k[l, m - 1], u_k[l, m], u_k[l, m + 1], r_grid[l], tau, h_phi, phi_grid[m], phi_next, phi_prev)
                    A[m, m + 1] = a_m(u_k[l - 1, m], u_k[l, m], u_k[l + 1, m], r_grid[l], tau, h_phi, phi_grid[m], phi_next, phi_prev)
                    A[m,     m] = 1 - A[m, m - 1] - A[m, m + 1]
                    d[m]        = u_tmp[l, m]
                u[l, :] = solution(A, d)

            u[ 0, :] = u_bound_r_left (t, phi_grid)
            u[-1, :] = u_bound_r_right(t, phi_grid)
    
            # проверка условия перехода на следующий временной слой
            cond_max = 0
            for l in range(1, L - 1, 1):
                for m in range(1, M - 1, 1):
                    cond_max = max(cond_max, abs((u[l, m] - u_k[l, m]) / u[l, m]))
            if cond_max < eps:
                flag_k = False
    
    u = u[::koef_L,::koef_M]
    

    return u


In [ ]:
koef_L = 16
koef_M = 16
N = 1281

u = progonka(koef_L, koef_M, N)


r_grid, h_r = np.linspace(0, 1, 6, retstep=True)
phi_grid, h_phi = np.linspace(0, np.pi / 2, 6, retstep=True)
u_an    = np.full((6, 6), np.nan)
delta_u = np.full((6, 6), np.nan)
norma = 0

for l in range(6):
    for m in range(6):
        u_an[l, m]   = u_analytical_1(1, r_grid[l], phi_grid[m])
        delta_u[l, m] = abs(u_an[l, m] - u[l, m])
        norma = max(norma, delta_u[l, m])

u = pd.DataFrame(u)
u_an = pd.DataFrame(u_an)


u_new = np.zeros((6,6))
u_new = u.astype(str)
for j in range(6):
    for i in range(6):
        u_new[i][j] = str("{:.4e}".format(u_an[i][j])) + " | " + str("{:.4e}".format(u[i][j]))
u_df = pd.DataFrame({**{"r/teta": r_grid}, **{phi_grid[i]:u_new[i] for i in range(6)}})
print("Первое число - аналитическое решение, второе - численное.")
display(u_df)

print("\033[1mМаксимальная норма ошибки =\033[0m", "\033[1m{:.4e}\033[0m".format((max(map(max, delta_u)))))
delta_df = pd.DataFrame({**{"r/teta": r_grid}, **{phi_grid[i]:delta_u[i] for i in range(6)}})
display(delta_df)

L =  81
M =  81


In [407]:
5.1152e-02/5.0769e-02

1.007543973684729

In [408]:
# print(f"u = \n{u}")
# print(f"u_an = \n{u_an}")
# print(f"delta_u = \n{delta_u}")
# print(f"norma = {norma}")